# TODO:
- NB-SVM
- Check how scoring works 

# Introduction
There are two major components we need to consider when it comes to Multi-label Classifiers ([source](https://users.ics.aalto.fi/jesse/talks/Multilabel-Part01.pdf)):
1. Methods for Multi-label Classfication:
    - Problem Transformation
    - Algorithm Adaptation
2. Multi-label Evaluation

## 1. Methods for Multi-label Classification
### a) Problem Transformation:
- Transforms the multi-label problem into single-label problem(s)
- Use any off-the-shelf single-label classifier to suit requirements
- i.e., Adapt your data to the algorithm

At training time:
1. Transform the multi-label training data to single-label data
2. Learn from the single-label transformed data

At testing time:
1. Make single-label predictions
2. Translate these into multi-label predictions

Examples:
1. Binary Relevance: $L$ binary problems (one vs. all):
2. Label Powerset: one multi-class problem of $2L$ class-values
3. Pairwise: $L(L−1)/2$ binary problems (all vs. all)
4. Copy-Weight: one multi-class problem of $L$ class values
5. [Classifier Chains](https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff)

### Comment:
- Some algorithms (like BR) do not model **label dependency!**
- The label ordering issue: Label ordering matters in algorithms such as Classifier chains. (There are some DL architectures that try to alleviate this problem e.g. [see](http://proceedings.mlr.press/v101/yang19b/yang19b.pdf))

### b) Algorithm Adaptation Methods:
- Adapt a single-label algorithm to produce multi-label outputs
- Benefit from specific classifier advantages (e.g., efficiency)
- i.e., Adapt your algorithm to the data

Examples: Adapt 
1. k-Nearest Neighbours
2. Decision Trees
3. Neural Networks
4. Support Vector Machines
5. Ensemble methods (Random Forest, XGboost, etc)

to learn from multi-label data and make multi-label predictions.

## 2. Multi-label Evaluation 
[source](https://www.researchgate.net/profile/Mohammad_Sorower/publication/266888594_A_Literature_Survey_on_Algorithms_for_Multi-label_Learning/links/58d1864392851cf4f8f4b72a/A-Literature-Survey-on-Algorithms-for-Multi-label-Learning.pdf)

1. **Hamming Loss:** it is the fraction of labels that are incorrectly predicted, i.e., the fraction of the wrong labels to the total number of labels.
\begin{equation}
HL = \frac{1}{N\cdot L}\sum_{i=1}^{N} \sum_{j=1}^L \mathcal{I}\left\{\hat y^{(i)}_j \neq y^{(i)}_j\right\}
\end{equation}

$\to$ for each instance compares each predicted label to actual label and counts the number of misclassified labels

2. **0/1 loss (a.k.a. Exact Match Ratio):** It is the most strict metric, indicating the percentage of samples that have all their labels classified correctly.
\begin{equation}
L_{0/1} =  \frac{1}{N}\sum_{i=1}^{N}  \mathcal{I}\left\{\hat{\textbf{y}}^{(i)} \neq \textbf{y}^{(i)}\right\}
\end{equation}

$\to$ for each instance compares all the tpredicted label to the actual ones and counts the number of misclassified instances

Note: There is a function in scikit-learn which implements subset accuracy, called ```accuracy_score```.
3. **Accuracy:** : Accuracy for each instance is defined as the proportion of the predicted correct labels
to the total number (predicted and actual) of labels for that instance. Overall accuracy is the average
across all instances
\begin{equation}
Accuracy = \frac{1}{N}\sum_{i=1}^N \frac{|\hat{\textbf{y}}^{(i)} \cap {\textbf{y}}^{(i)} |}{|\hat{\textbf{y}}^{(i)} \cup {\textbf{y}}^{(i)} |}
\end{equation}


4. **Log Loss:** Sometimes we want to evaluate probabilities to encourage good ‘confidence’. Log Loss takes into account the uncertainty of your prediction based on how much it varies from the actual label. This gives us a more nuanced view into the performance of our model.
\begin{equation}
\text{Log-Loss} = - \sum_{i=1}^N\sum_{j=1}^L y_j^{(i)} \log \left(\hat{y}_j^{(i)}\right)
\end{equation}

5. **Ranking Loss:** to encourage good ranking; ranking loss evaluates the average proportion of label pairs that are incorrectly ordered for an instance.

Other metrics used in the literature:
- one error – if top ranked label is not in set of true labels
- coverage – average “depth” to cover all true labels
- precision
- recall
- macro-averaged F1 (ordinary averaging of a binary measure)
- micro-averaged F1 (labels as different instances of a ‘global’ label)
- precision vs. recall curves

### Comments:
- Hamming loss can in principal be minimized without taking label dependence into account.
- For 0/1 loss label dependence must be taken into account.
- Possible thresholding strategies:
    1. Select a threshold $t$ from an internal validation test, e.g. $t\in\{0.1, 0.2, . . . , 0.9\}$ $\to$ slow!!!
    2. Calibrate a threshold such that $LCard({\textbf{y}}) ≈ LCard(\hat{\textbf{y}} )$
       - e.g., training data has label cardinality of 1.7;
       - set a threshold $t$ such that the label cardinality of the test data is as close as possible to 1.7

    3. Calibrate $L$ thresholds such that each $LCard(y_j) ≈ LCard(\hat{y}_j)$
       - e.g., the frequency of label $y_j = 1$ is 0.3,
       - set a threshold $t_j$ such that $h_j(\textbf{x}) \geq t_j \leftrightarrow y_j = 1$ with frequency as close as possible to 0.3

# Methodology
## 1. Training 
### a) Problem Transformation Algorithms:
Train the following models:
1. Binary Relevance (does not model label dependencies)
2. Label Powerset (models label dependencies)
3. Classifier chains (models label dependencies)

with:
- NB
- SVM
- LogReg
- NB-SVM

### b) Algorithm Adaptation Methods:
Train the following models:
1. kNN
2. Trees
3. DL

## 2. Evaluation:
1. HL
2. LogLoss
3. Exact Match Ratio
4. AUC score
5. Micro/Macro F1-score
6. Rank loss

In [435]:
# dataframes and arrays
import pandas as pd
import numpy as np
from time import time
import warnings
warnings.simplefilter('ignore')

# sklearn models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.dummy import DummyClassifier

# sklearn metrics
from sklearn.metrics import accuracy_score, log_loss, hamming_loss, label_ranking_loss
from sklearn.metrics import multilabel_confusion_matrix, roc_auc_score, f1_score
from sklearn.metrics import make_scorer

# sklearn tools
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted, check_array
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier # also implements multilabel clf just like MultiOutputClassifier (BR)
from sklearn.multioutput import MultiOutputClassifier 
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler, Binarizer
from sklearn.utils.extmath import safe_sparse_dot

# skmulilearn models/wrappers (for multi-label classification)
from skmultilearn.problem_transform import BinaryRelevance, LabelPowerset, ClassifierChain

from scipy import sparse

# utilities
from utilities.constants import CLASSES
from utilities.functions import load_objects

# directories
DATA_DIR = './data/'


In [481]:
# Helper Functions
log_loss_scorer = make_scorer(log_loss, greater_is_better=False)

In [2]:
X_train_tfv = load_objects(DATA_DIR + 'X_train_tfv')
y = load_objects(DATA_DIR + 'y')

In [3]:
X_train_tfv.shape, y.shape

((159571, 30000), (159571, 6))

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X_train_tfv, y, random_state=42, test_size=0.20, shuffle=True)

In [5]:
for name, data in zip(['X_train', 'X_val', 'y_train', 'y_val'],[X_train, X_val, y_train, y_val]):
    print(name, data.shape)

X_train (127656, 30000)
X_val (31915, 30000)
y_train (127656, 6)
y_val (31915, 6)


# Speed Tests:

In [147]:
clfs = {'OvR': OneVsRestClassifier(MultinomialNB()),
        'MultiOutput': MultiOutputClassifier(MultinomialNB()),
        'BinaryRelevance': BinaryRelevance(MultinomialNB())
       }
for name, clf in clfs.items():

    start_time = time()
    clf.fit(X_train, y_train)
    loss = log_loss(y_val, clf.predict(X_val))
    end_time = time()
    print(name)
    print(f"loss: {loss}")
    print("time: {} sec".format(end_time-start_time))

OvR
loss: 0.3490358005186171
time: 0.26319313049316406 sec
MultiOutput
loss: 0.3490358005186171
time: 0.2046949863433838 sec
BinaryRelevane
loss: 0.3490358005186171
time: 369.0491449832916 sec


The sklearn models are identical when it comes to **multi-label classification** and in turn, they are identical to BinaryRelevance. MultiOutput appears to be slighlty faster than OvR, while BR is substantially slower than the sklearn models. Henceforth, we will be using the MultiOutput wrapper to implement binary relevance.

# 0. Dummy Classifier

In [469]:
dummy_clf = MultiOutputClassifier(DummyClassifier(strategy='most_frequent'))
dummy_clf.fit(X_train, y_train)
log_loss(y_val, dummy_clf.predict(X_val))

0.39714574605418335

# 1. Multinomial Naive Bayes

In [478]:
nb_params = [
    {
        'estimator__alpha': [0.001, 0.01, 0.1, 1.0]
    }]

In [482]:
nb_br = GridSearchCV(MultiOutputClassifier(MultinomialNB()), nb_params, scoring=log_loss_scorer, n_jobs=-1, cv=5)
nb_br.fit(X_train_tfv, y)
nb_br.best_score_

-1.2681982219918218

# 2. Logistic Regression 

In [264]:
lr_pipe = Pipeline([('svd', TruncatedSVD(n_components=50)),
                    ('scaler', StandardScaler()),
                    ('lr', MultiOutputClassifier(LogisticRegression()))
                   ])

In [265]:
lr_params = [
    {
        'svd__n_components': [50, 100, 150],
        'lr__estimator__C': [10**n for n in range(-3,3) ],
        'lr__estimator__penalty': ['l1', 'l2']
    }
]

In [267]:
print("LogRegBR")
print("log_loss: {}".format(lr_br.best_score_))
print("time: {} sec".format(end_time-start_time))

LogRegBR
log_loss: -1.1483635067766411
time: 3706.1563770771027 sec


In [268]:
lr_br.best_params_

{'lr__estimator__C': 0.001,
 'lr__estimator__penalty': 'l1',
 'svd__n_components': 50}

# 3. SVM

In [295]:
svm_pipe = Pipeline([('svd', TruncatedSVD(n_components=10)),
                    ('scaler', StandardScaler()),
                    ('svm', MultiOutputClassifier(LinearSVC(random_state=12)))
                   ])

In [299]:
svm_params = [
    {
        'svd__n_components': [20,50],
        'svm__estimator__C': [10**n for n in range(-3,1)],
        'svm__estimator__penalty': ['l2'],
        'svm__estimator__tol': [10**n for n in range(-5,-2)]
    }
]

In [300]:
start_time = time()
svm_br = GridSearchCV(svm_pipe, svm_params, scoring=log_loss_scorer, n_jobs=-1, cv=3)
svm_br.fit(X_train_tfv, y)
end_time = time()

In [301]:
print("SVM")
print("log_loss: {}".format(svm_br.best_score_))
print("time: {} sec".format(end_time-start_time))

SVM
log_loss: -1.407751627034678
time: 1211.24240899086 sec


In [302]:
svm_br.best_params_

{'svd__n_components': 20,
 'svm__estimator__C': 0.001,
 'svm__estimator__penalty': 'l2',
 'svm__estimator__tol': 1e-05}

In [475]:
roc_auc_score(y_val, svm_br.predict(X_val))

0.5910059878314433

# 4. NB-SVM

Note:
1. a Multinomial Naive Bayes classifier is a **linear (binary) classifier** ([proof](https://svivek.com/teaching/lectures/slides/naive-bayes/naive-bayes-linear.pdf)):

\begin{align}
y^{(i)} &= \text{sign}\left(\textbf{w}^T \textbf{x}^{(i)}+b\right)\\
\textbf{w} &= \log \left(\frac{\textbf{p}/||\textbf{p}||_1}{\textbf{q}/||\textbf{q}||_1}\right),\quad  &&b= \log\left(\frac{\sum_i \mathcal{I}\left\{ y^{(i)}=1 \right\} }{\sum_i \mathcal{I}\left\{ y^{(i)}=-1 \right\}}\right)\\
\textbf{p} &= \alpha + \sum_{i} \textbf{x}^{(i)}\cdot \mathcal{I}\left\{ y^{(i)}=1 \right\} ,\quad &&\textbf{q} = \alpha + \sum_{i} \textbf{x}^{(i)} \cdot \mathcal{I}\left\{ y^{(i)}=-1 \right\}
\end{align}

2. SVM with NB features ([NBSVM](https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf)):
\begin{align}
y^{(i)} &= \text{sign}\left({\textbf{w}^\prime}^T {\textbf{x}^\prime}^{(i)}+b\right)\\
{\textbf{x}^\prime}^{(i)} & = \textbf{r} \circ \textbf{x}^{(i)} \quad \text{with} \quad \textbf{r} = \log \left(\frac{\textbf{p}/||\textbf{p}||_1}{\textbf{q}/||\textbf{q}||_1}\right)\\
\textbf{w}^\prime & = (1-\beta) \,\bar{w} + \beta \,\textbf{w}
\end{align}

Advanced Tips:
1. **Binarization:**
 - Binarize features such that:  $\qquad\hat{\textbf{x}}^{(i)} = \mathcal{I}\left\{ \textbf{x}^{(i)}>0\right\}$. 
 - Calculate $\textbf{p}, \textbf{q}, \textbf{r}$ using $\hat{\textbf{x}}^{(i)}$.


2. **The Interpolation Parameter ($\beta$):**
 - Best to set $\beta$ in the range of $\left[1/4, 1/2\right]$. 
 - To avoid further hyperparameter tuning, set $\beta = 1$ s.t. $\textbf{w}^\prime = \textbf{w}$.




In [432]:
class NbSvmClassifier(BaseEstimator, ClassifierMixin):
    
    def __init__(self, C=1.0, alpha=1.0, beta=1.0, dual=False, n_jobs=1):
        self.C = C
        self.alpha = alpha
        self.beta = beta
        self.dual = dual
        self.n_jobs = n_jobs


    def _pr(self, X, y_i, y):
        
        p = X[y==y_i].sum(axis=0)
        
        return (self.alpha + p) / (self.alpha + (y==y_i).sum())

    def fit(self, X, y):
        
        # Check that X and y have correct shape
        X, y = check_X_y(X, y, accept_sparse=True)

        # Create NB features 
        self._r = sparse.csr_matrix(np.log(self._pr(X, 1, y) / self._pr(X, 0, y)))
        X_nb = X.multiply(self._r)
        
        # Train a Logistic Regression
        self._clf = LogisticRegression(C=self.C, dual=self.dual, n_jobs=self.n_jobs).fit(X_nb, y)
        
        # Define the scaled/shifted weight coefficients, intercept, classes
        self.coef_ = (1 - self.beta) * np.mean(self._clf.coef_) + self.beta * self._clf.coef_
        self.intercept_ = self._clf.intercept_
        self.classes_ = self._clf.classes_
        
        return self
    
    def decision_function(self, X):

        X = check_array(X, accept_sparse='csr')
        
        n_features = self.coef_.shape[1]
        if X.shape[1] != n_features:
            raise ValueError("X has %d features per sample; expecting %d"
                             % (X.shape[1], n_features))

        scores = safe_sparse_dot(X, self.coef_.T,
                                 dense_output=True) + self.intercept_
        return scores.ravel() if scores.shape[1] == 1 else scores
    
    
    def predict(self, X):
        
        X_nb = X.multiply(self._r)
        
        scores = self.decision_function(X_nb)
        
        if len(scores.shape) == 1:
            indices = (scores > 0).astype(np.int)
        else:
            indices = scores.argmax(axis=1)
            
        return self.classes_[indices]



In [455]:
nbsvm_pipe = Pipeline([('svd', TruncatedSVD(n_components=200)),
                       ('bin', Binarizer())
                      ])

In [457]:
X_train_bin = Binarizer().fit_transform(X_train)
X_val_bin = Binarizer().fit_transform(X_val)

In [485]:
best_loss = 100
for beta in [0.25, 0.5, 0.75, 1]:
    for C in [0.1, 1, 5, 10]:
        nbsvm_clf = MultiOutputClassifier(NbSvmClassifier(C=C, beta=beta, dual=True, n_jobs=-1)).fit(X_train_bin, y_train)
        y_preds = nbsvm_clf.predict(X_val_bin)
        loss = -log_loss(y_val, y_preds)
        print({'beta': beta, 'C': C})
        print(loss)
        print(roc_auc_score(y_val, y_preds))
        print('\n')
        if loss < best_loss:
            best_loss = loss
            best_clf = nbsvm_clf
            best_params = {'beta': beta, 'C': C}

{'beta': 0.25, 'C': 0.1}
-1.5510199858351847
0.5275580051848201


{'beta': 0.25, 'C': 1}
-1.9510188547657694
0.5475807806002264


{'beta': 0.25, 'C': 5}
-2.038474825754459
0.5698241749687063


{'beta': 0.25, 'C': 10}
-2.0934426836777527
0.5801653347225113


{'beta': 0.5, 'C': 0.1}
-1.9300572390883368
0.5850152159434782


{'beta': 0.5, 'C': 1}
-2.005808811500884
0.6131079997756285


{'beta': 0.5, 'C': 5}
-2.0026344640216296
0.6313964645766436


{'beta': 0.5, 'C': 10}
-1.9818000417223052
0.6383058998807447


{'beta': 0.75, 'C': 0.1}
-1.8165866551139214
0.6480771896155139


{'beta': 0.75, 'C': 1}
-1.790321183894278
0.6766445181476682


{'beta': 0.75, 'C': 5}
-1.787914138577026
0.6935189544058692


{'beta': 0.75, 'C': 10}
-1.7729873772589277
0.699735228047197


{'beta': 1, 'C': 0.1}
-1.5053393861506121
0.7127344857056616


{'beta': 1, 'C': 1}
-1.5155525857834675
0.7360703889668305


{'beta': 1, 'C': 5}
-1.5499886943702532
0.7457686457044129


{'beta': 1, 'C': 10}
-1.5751757904004289
0.7486